
<h1><center>Bài giới thiệu một số cách so sánh trong Stata
    
</center></h1>

Một trong những vấn đề thường gặp trong quản lý dữ liệu là so sánh những giá trị khác nhau của cùng 1 biến (So sánh giá trị ở 2 hàng khác nhau). Những vấn đề này gặp phổ biến hơn trong dữ liệu mảng hoặc dữ liệu theo dõi dọc.

Bài giới thiệu này sẽ đưa ra một số giải pháp để giải quyết vấn đề qua từng ví dụ cụ thể với hàm sum() và cond()

Tài liệu tham khảo: https://journals.sagepub.com/doi/pdf/10.1177/1536867X1101100210

## Ví dụ 1: Bộ số liệu về tuổi thọ trung bình (life expectancy) của Mỹ từ năm 1900.

In [1]:
sysuse uslifeexp, clear
describe, short


(U.S. life expectancy, 1900-1999)


Contains data from C:\Program Files (x86)\Stata15\ado\base/u/uslifeexp.dta
  obs:           100                          U.S. life expectancy, 1900-1999
 vars:            10                          30 Mar 2016 04:31
 size:         3,800                          
Sorted by: year


In [2]:
%head 5

,year,le,le_male,le_female,le_w,le_wmale,le_wfemale,le_b,le_bmale,le_bfemale
1,1900,47.299999,46.299999,48.299999,47.599998,46.599998,48.700001,33,32.5,33.5
2,1901,49.099998,47.599998,50.599998,49.400002,48,51,33.700001,32.200001,35.299999
3,1902,51.5,49.799999,53.400002,51.900002,50.200001,53.799999,34.599998,32.900002,36.400002
4,1903,50.5,49.099998,52,50.900002,49.5,52.5,33.099998,31.700001,34.599998
5,1904,47.599998,46.200001,49.099998,48,46.599998,49.5,30.799999,29.1,32.700001


Bộ dữ liệu này chứa giá trị về kỳ vọng sống ở Mỹ qua nhiều năm, tính từ năm 1900.

Chú thích: _le_ là viết tắt của _life expectancy_

Giả sử: muốn tính thay đổi tuổi thọ trung bình lấy năm 1960 làm tham chiếu. Có nhiều cách tính khác nhau cho trường hợp này.

In [3]:
// Cách tính đầu tiên:
* Để ý rằng bộ số liệu đang xếp theo thứ tự tăng dần theo năm, do đó năm 2960 sẽ ở record 61
* tính sự thay đổi trong tuổi thọ trung bình so với năm 1960
gen le_index = 100 * le/le[61] // đơn vị %
list year le le_index in 55/64




     +------------------------+
     | year     le   le_index |
     |------------------------|
 55. | 1954   69.6   99.85653 |
 56. | 1955   69.6   99.85653 |
 57. | 1956   69.7        100 |
 58. | 1957   69.5   99.71306 |
 59. | 1958   69.6   99.85653 |
     |------------------------|
 60. | 1959   69.9   100.2869 |
 61. | 1960   69.7        100 |
 62. | 1961   70.2   100.7174 |
 63. | 1962   70.1   100.5739 |
 64. | 1963   69.9   100.2869 |
     +------------------------+


Như vậy chúng ta có thể thấy được sự thay đổi tuổi thọ trung bình (theo %) với mốc là năm 1960.

Tuy nhiên với phương pháp này thì khi dữ liệu không sắp xếp tăng dần theo năm thì giá trị tuổi thọ trung bình của năm 1960 sẽ không còn nằm ở hàng thứ 61. Do đó, áp dụng cách này sẽ dễ gây nhầm lẫn.

In [4]:
// Cách thứ 2: chắc chắn là gán đúng giá trị của năm 1960
* tạo biến mới để đánh dấu giá trị năm 1960
gen tag = year==1960
sort tag 
* sau khi sort thì giá trị tuổi thọ của năm 1960 luôn nằm ở hàng cuối cùng
gen le_index2 = 100 * le/le[_N]
assert le_index == le_index2

In [5]:
// Cách thư 3: Cách tổng quát hơn kể cả đối với số liệu mảng và chuỗi thời gian.
gen basevalue = sum((year==1960) * le)
gen le_index3 = 100 * le/le[_N]
assert le_index == le_index3

Tại sao cách thứ 3 là cách tổng quát, bao gồm trường hợp số liệu mảng và chuỗi thời gian. 
Ở đây sẽ không bàn về phương pháp thứ nhất, bởi với mỗi nhóm thì giá trị cgần so sánh có thể ở hàng khác nhau

Bằng bộ số liệu dưới đây, chúng ta sẽ thấy tại sao cách 2 không phải là cách tổng quát.

In [6]:
webuse grunfeld, clear

In [7]:
%head 5

,company,year,invest,mvalue,kstock,time
1,1,1935,317.60001,3078.5,2.8,1
2,1,1936,391.79999,4661.7002,52.599998,2
3,1,1937,410.60001,5387.1001,156.89999,3
4,1,1938,257.70001,2792.2,209.2,4
5,1,1939,330.79999,4313.2002,203.39999,5


In [8]:
%tail 5

,company,year,invest,mvalue,kstock,time
196,10,1950,3.4200001,69.050003,8.7399998,16
197,10,1951,4.6700001,83.040001,9.0699997,17
198,10,1952,6,74.419998,9.9300003,18
199,10,1953,6.5300002,63.509998,11.68,19
200,10,1954,5.1199999,58.119999,14.33,20


In [9]:
// Áp dụng câu lệnh tương tự với cách làm thứ 2
* tính thay đổi % invest với mốc thời gian là năm 1939, theo từng công ty (company)
gen byte baseyear = 1 if year == 1939
bysort company (baseyear): gen invest_index = 100 * invest/invest[1]


(190 missing values generated)



Nếu để ý kĩ hơn vào câu lệnh thì đối với những công ty (company) nào mà không có số liệu cho năm 1939 thì giá trị của baseyear sẽ bằng missing hết. Do đó, giá trị invest_index của công ty đó sẽ so sánh với năm ở hàng đầu tiên.

Đối với những dữ liệu thật, chúng ta phải đảm bảo rằng câu lệnh của chúng ta bao gồm hết tất cả các trường hợp. Cùng phân tích kĩ hơn câu lệnh theo cách số 3

In [10]:
bysort company: gen basevalue = sum((year==1939) * invest)
by company: gen invest_index2 = 100 * (invest/basevalue[_N])

Thay vì đánh dấu năm mong muốn, chúng ta đánh dấu giá trị của năm mong muốn (basevalue thay vì baseyear)
(year==1939) trong lệnh sum có giá trị bằng 0 hoặc bằng 1, tuỳ thuộc vào điều kiện trong ngoặc có xảy ra hay không.
Do đó giá trị của basevalue sẽ là 0 hoặc giá trị của invest ở năm 1939
đặc điểm của sum() là luôn để giá trị lớn nhất ở cuối cùng do đó không cần sort.

Như vậy khi tính invest_index2 sẽ bao quát hết tất cả trường hợp của các nhóm kể cả khi 1 công ty nào đó không có số liệu theo dõi cho năm 1939.

_Chú ý:_ Hàm total() có thể dùng thay cho hàm sum() để cho kết quả tương tự nhưng dùng với egen. Câu lệnh như sau:

In [11]:
cap drop basevalue
bysort company: egen basevalue = total((year==1939) * invest)
by company: gen invest_index3 = 100 * (invest/basevalue[_N])

# Ví dụ 2: Sử dụng bộ số liệu auto với hàm cond()

In [12]:
sysuse auto, clear

(1978 Automobile Data)


Với mỗi nhóm foreign thì tính giá trị trung bình của weight với điều kiện là mpg > 25.

In [13]:
// Cách 1: tính tuần tự theo các bước
bysort foreign: egen mean_weight = mean(weight) if mpg>25 

(60 missing values generated)


Câu lệnh trên tính trung bình của cân nặng (weight) theo từng nhóm foreign với điều kiện mpg>25. 
Tuy nhiên, với những trường hợp mà mpg<25 thì mean_weight sẽ là giá trị missing.
Do đó phải thay thế giá trị missing này bằng giá trị của nhóm foreign để tạo ra biến hoàn chỉnh.

In [14]:
bysort foreign (mean_weight): replace mean_weight = mean_weight[1]

(60 real changes made)


In [15]:
// Cách 2: Sử dụng hàm cond() cho kết quả tương tự với 1 dòng lệnh
bysort foreign: egen mean_weight2 = mean(cond(mpg > 25, weight, .))

In [16]:
assert mean_weight==mean_weight2

Lệnh assert đã cho thấy 2 cách làm có kết quả như nhau.

In [17]:
// Cách 3: Sử dụng điều kiện trong 1 thương số
bysort foreign: egen mean_weight3 = mean(weight/(mpg>25))
assert mean_weight==mean_weight3

Ở đây mpg>25 là một điều kiện trả về giá trị 0 hoặc 1. Khi đó với mỗi nhóm foreign tính mean weight sẽ là tính mean khi mpg>25. Ở đây để ý rằng khi tính mean thì hàm sẽ tự động loại bỏ giá trị missing, chỉ tính những giá trị non-missing và trả về giá trị tính được cho tất cả các hàng.

In [18]:
// Tương tự như cách 3 chúng ta có thể sử dụng để xem xe nào có giá nhỏ nhất với mpg>25
bysort foreign: egen min_price = min(price/(mpg>25))

In [ ]:
// Bằng cách tương tự như trên, chúng ta cũng có thể tính được người trưởng thành có tuổi nhỏ nhất.
// Ví dụ: Theo câu lệnh dưới đây (câu lệnh giả định)
bysort family: egen youngest_audult = min(age/(age>18))

# Kết luận:


Bài trên đây chúng ta đã đi qua một số phương pháp so sánh các giá trị giữa các hàng trong Stata. Đây là một góc nhìn mới ngay cả với những người đã sử dụng Stata một thời gian dài. Bài giới thiệu thú vị này hi vọng sẽ có ích cho việc thực hành Stata của các bạn. Chúc các bạn thực hành thành công!